In [29]:
%reload_ext autoreload
%autoreload 2
from termcolor import colored
import dotenv
import sys
import dspy
import os

sys.path.append('../pipeline_v2/')
import main 
dotenv.load_dotenv('../.env')

from utils import print_header

# main.VERDICTS=["Supported", "Refuted", "Not Enough Evidence", "Conflicting Evidence/Cherry-picking"]

# Initialize search provider
main.NUM_SEARCH_RESULTS = 10 # Number of search results to retrieve
main.SCRAPE_TIMEOUT = 5 # Timeout for scraping a webpage (in seconds)
search_provider = main.SearchProvider(provider="duckduckgo")

# Initialize DSPy
lm = dspy.LM('gemini/gemini-1.5-flash', api_key=os.getenv('GOOGLE_GEMINI_API_KEY'))
#lm = dspy.LM('ollama_chat/mistral', api_base='http://localhost:11434', api_key='')
dspy.settings.configure(lm=lm)

# Initialize pipeline
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
main.VERBOSE = True # Print intermediate results
main.INTERACTIVE = False # Allow the user to provide feedback
main.USE_BM25 = True # Use BM25 for retrieval (in addition to cosine similarity)
main.BM25_WEIGHT = 0.5 # Weight for BM25 in the hybrid retrieval

pipeline = main.FactCheckPipeline(
    search_provider=search_provider,
    model_name=lm,
    embedding_model=embedding_model,
    retriever_k=2
)

# Example statement to fact-check
# statement = """And then there's the reality of the Trump economy, 
# where wages adjusted for inflation were rising. The wage gap between 
# rich and poor was shrinking. The savings rate for black Americans was 
# the highest in the history of our country."""

In [31]:
statement = """"Statement: Crime is down in Venezuela by 67% because they're taking their gangs and their criminals and depositing them very nicely into the United States. \n Source: Donald Trump \n Date: 2024-04-02”"""
verdict, confidence, reasoning, claims = pipeline.fact_check(statement)

# Print final result
print("\nFinal Fact-Check Result:")
print_header(f"Statement: {colored(statement, 'white')}", level=1)
print_header(f"Overall Verdict: {colored(verdict, 'green')}", level=1)
print_header(f"Overall Confidence: {colored(str(confidence), 'yellow')}", level=1)
print_header(f"Overall Reasoning: {colored(reasoning, 'cyan')}", level=1)
# print(result)
# print(json.dumps(result, indent=2))

===== Starting Fact Check Pipeline =====
 Original Statement: "Statement: Crime is down in Venezuela by 67% because they're taking their gangs and their criminals and depositing them very nicely into the United States. 
 Source: Donald Trump 
 Date: 2024-04-02” 
  ===== Atomic Claim Extraction =====
   Extracted Claims (2):  
     1. Donald Trump stated on 2024-04-02 that crime in Venezuela decreased by 67%. 
     2. According to Donald Trump on 2024-04-02, the decrease in Venezuelan crime is attributed to the relocation of gangs and criminals to the United States. 
    ===== Claim Decomposition [1/2] =====
       Decomposed Components (Questions + Search Queries) (2): 
         1. Question: What were the official crime statistics (e.g., homicide rates, violent crime rates) in Venezuela before and after April 2, 2024? 
            Search Queries: ['Venezuela crime statistics 2023-2024', 'Venezuela official government crime data 2024'] 
         2. Question: Did any reputable internatio

        ===== Web Search for Query [2/2] =====
         Query: Venezuela official government crime data 2024 
         Retrieved 10 Sources: 
           1. Why Is Venezuela's Crime Rate Falling? - InSight Crime 
           URL: https://insightcrime.org/news/venezuela-crime-rate-falling/ 
           Excerpt: A government-reported decline in Venezuela's crime rate may create the illusion of a security breakthrough, but the reduction has more to do with reconfigurations in the country's underworld than an effective state response. In mid-May, Venezuelan security officials announced that crime indicators had fallen by 25.1% compared to 2023. The Venezuelan government attributes the decrease to ... 
           2. Crime and violence in Venezuela - statistics & facts | Statista 
           URL: https://www.statista.com/topics/12670/crime-and-violence-in-venezuela/ 
           Excerpt: Discover all statistics and data on Crime and violence in Venezuela now on statista.com! 
           3. Polit

      ===== Synthesizing Answer [1/2] =====
         Question: What were the official crime statistics (e.g., homicide rates, violent crime rates) in Venezuela before and after April 2, 2024? 
         Search Queries: ['Venezuela crime statistics 2023-2024', 'Venezuela official government crime data 2024'] 


         Answer: I'm sorry, but the provided documents do not contain specific official crime statistics (homicide rates, violent crime rates) for Venezuela before and after April 2, 2024. While the documents mention a decrease in intentional homicides compared to previous years [3], they lack precise numerical data and specific dates to answer your question. 
         Citations:  
      ===== Question Answering for Component [2/2] =====
         Question: Did any reputable international organizations or news sources report on changes in Venezuelan crime rates around April 2, 2024, corroborating or contradicting a 67% decrease? 
         Search Queries: ['Venezuela crime rate reports April 2024', 'International organizations Venezuela crime data 2024'] 
        ===== Web Search for Query [1/2] =====
         Query: Venezuela crime rate reports April 2024 
         Retrieved 10 Sources: 
           1. PolitiFact | Donald Trump exaggerates Venezuelan crime drop and ... 
           URL: h

        ===== Web Search for Query [2/2] =====
         Query: International organizations Venezuela crime data 2024 


...

         Answer: While several sources report a rise in Venezuelan gang members crossing the US-Mexico border and causing concern among law enforcement [3], there is no evidence presented to support or refute a significant relocation around April 2, 2024.  Reports indicate ongoing investigations into over 100 cases involving Venezuelan gang members [3], and the arrest and subsequent release of four gang members in Colorado between 2022 and 2023 [2].  However, these reports do not provide data on the timing or scale of migration specifically around April 2, 2024. 
         Citations:  
           [1] Law enforcement officials, including from the NYPD, are warning about a surge of Venezuelan gang members crossing the U.S.-Mexico border. There are now more than 100 ongoing investigations ... 
           Source: Officials warn about rising threat from Venezuelan gang members ... (https://www.nbcnews.com/nightly-news/video/officials-warn-about-rising-threat-from-venezuelan-gang-members-ille

        ===== Web Search for Query [2/2] =====
         Query: Donald Trump statements April 2 2024 Venezuela 


         Retrieved 10 Sources: 
           1. Donald Trump says Venezuela agreed to accept migrants back, lauds ... 
           URL: https://thehill.com/policy/international/5120808-trump-venezuela-hostages-migrants/ 
           Excerpt: President Trump said that Venezuela has agreed to accept the country's migrants living in the U.S. back as he lauded the return of six American hostages who were held in the Latin American ... 
           2. Senior Trump official travels to Venezuela for talks on migrants 
           URL: https://www.pbs.org/newshour/world/senior-trump-official-travels-to-venezuela-for-talks-on-migrants 
           Excerpt: Mauricio Claver-Carone, Trump's special envoy to Latin America, confirmed Grenell's visit to Caracas in a conference call with journalists on Friday. 
           3. Senior Trump official travels to Venezuela to discuss migrants with ... 
           URL: https://apnews.com/article/trump-maduro-envoy-migrants-grenell-edfa58920c0a76f0bbe9de995a07d1a3 


      ===== Synthesizing Answer [3/3] =====
         Question: Did Donald Trump make a statement on April 2, 2024, regarding Venezuelan crime and its relation to the United States? 
         Search Queries: ['"Donald Trump" "Venezuela crime" "April 2 2024"', 'Donald Trump statements April 2 2024 Venezuela'] 


         Answer: Based on the provided sources, there is no evidence that Donald Trump made a statement on April 2, 2024, specifically addressing Venezuelan crime and its connection to the United States.  However, around April 10th, 2024, he did make statements about a Venezuelan crime drop, which were subsequently fact-checked and found to be exaggerations [3, 4]. 
         Citations:  
           [1] Donald Trump exaggerates Venezuelan crime drop and... 
           Source: PolitiFact | Donald Trump exaggerates Venezuelan crime drop and ... (https://www.politifact.com/factchecks/2024/apr/10/donald-trump/donald-trump-exaggerates-venezuelan-crime-drop-and/) 
           [2] Gobierno Bolivariano de Venezuela, Índice delictivo se reduce en un 32,4% en Venezuela en comparación con el 2023, March 18, 2024 Venezuelan Observatory of Violence, Annual Report Violence 2023 ... 
           Source: PolitiFact | Donald Trump exaggerates Venezuelan crime drop and ... (https://www.politifact.com/factc